In [66]:
import numpy as np
import tensorflow as tf
import collections

### Reading data and converting to bigrams

In [67]:
def read_data(file):
    with open(file, 'r') as f:
        text = f.read()
        text = text.replace('\n', '')
        start_idx = text.find('In Benares')
        end_idx = text.find('FOOTNOTES')
        text = text[start_idx:end_idx]
        text = text.lower().strip()
        
        bigram_text = [''.join(text[char:char + 2]) for char in range(0, len(text) - 2, 2)]
        
    return bigram_text

In [68]:
bigram_text = read_data('2400-0.txt')
print('no. of bigrams:', len(bigram_text))
bigram_text[0:10]

no. of bigrams: 162287


['in', ' b', 'en', 'ar', 'es', ' o', 'nc', 'e ', 're', 'ig']

### Creating Dictionary

In [69]:
def create_dict(bigrams):
    
    count = []
    count.extend(collections.Counter(bigrams).most_common())
    
    dictionary = dict({'UNK':0})
    for char, freq in count:
    
        dictionary[char] = len(dictionary)    
            
    rev_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return dictionary, rev_dictionary, count


In [70]:
dictionary, rev_dictionary, count = create_dict(bigram_text)
vocab_size = len(dictionary)

print('dictionary', list(dictionary)[:10])
print('reverse dictionary', list(rev_dictionary)[:10])
print('most common words:', count[0:5])
print('len of dictionary:', len(dictionary))

dictionary ['37', 'nu', 'mw', 'un', 'p,', 'pu', 'n)', 'b-', '?’', 's ']
reverse dictionary [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
most common words: [('e ', 4702), (' t', 4538), ('he', 4474), ('th', 4451), ('d ', 3014)]
len of dictionary: 1045


### Converting from bigrams to int

In [71]:
def bigram_to_int(bigram_text, dictionary):
    
    bigram_int = []
    
    for bigram in bigram_text:
        
        if bigram in dictionary:
            bigram_int.append(dictionary[bigram])
        else:
            bigram_int.append(dictionary['UNK'])

    return bigram_int

In [72]:
bigram_int = bigram_to_int(bigram_text, dictionary)
print(list(bigram_int[0:10]))

[9, 23, 24, 44, 34, 19, 118, 1, 17, 155]


### Generating batches

In [73]:
def next_batch(bigrams, batch_size, num_unroll):
    global vocab_size
    
    segments = len(bigrams) // batch_size
    
    cursor = [(offset * segments) + num_unroll for offset in range(batch_size)]
    
    
    batch_data = np.zeros((batch_size,vocab_size),dtype=np.float32)
    batch_labels = np.zeros((batch_size,vocab_size),dtype=np.float32)

    
    for b in range(batch_size):
        
        if cursor[b]+1>= len(bigrams):
            cursor[b] = b * segments
        
        batch_data[b, bigram_int[cursor[b]]] = 1.0
        batch_labels[b, bigram_int[cursor[b] + 1 ]] = 1.0
        
        cursor[b] = (cursor[b]+1)%len(bigrams)
    
    return batch_data, batch_labels

### Unrolling batches

In [74]:
def unroll_batches(bigram_int, num_unroll, batch_size):
    
    unroll_data = []
    unroll_labels = []
    
    for u_idx in range(num_unroll):
        
        batch_data, batch_labels = next_batch(bigram_int, batch_size, u_idx)
        
        unroll_data.append(batch_data)
        unroll_labels.append(batch_labels)
        
    return unroll_data, unroll_labels

In [75]:
unroll_data, unroll_labels = unroll_batches(bigram_int[:50] , 10, 5)
print('unroll data shape:', np.array(unroll_data).shape)
print('unroll labels shape:', np.array(unroll_labels).shape)

for ui,(dat,lbl) in enumerate(zip(unroll_data,unroll_labels)):   
    print('\n\nUnrolled index %d'%ui)
    dat_ind = np.argmax(dat,axis=1)
    lbl_ind = np.argmax(lbl,axis=1)
    print('\tInputs:')
    for sing_dat in dat_ind:
        print('\t%s (%d)'%(rev_dictionary[sing_dat],sing_dat),end=", ")
    print('\n\tOutput:')
    for sing_lbl in lbl_ind:        
        print('\t%s (%d)'%(rev_dictionary[sing_lbl],sing_lbl),end=", ")

unroll data shape: (10, 5, 1045)
unroll labels shape: (10, 5, 1045)


Unrolled index 0
	Inputs:
	in (9), 	ne (68), 	 b (23), 	t, (175), 	n  (15), 
	Output:
	 b (23), 	d  (5), 	y  (29), 	 t (2), 	va (216), 

Unrolled index 1
	Inputs:
	 b (23), 	d  (5), 	y  (29), 	 t (2), 	va (216), 
	Output:
	en (24), 	a  (59), 	na (176), 	o  (32), 	jr (437), 

Unrolled index 2
	Inputs:
	en (24), 	a  (59), 	na (176), 	o  (32), 	jr (437), 
	Output:
	ar (44), 	mi (142), 	me (57), 	wh (70), 	am (125), 

Unrolled index 3
	Inputs:
	ar (44), 	mi (142), 	me (57), 	wh (70), 	am (125), 
	Output:
	es (34), 	gh (132), 	 p (51), 	os (139), 	uk (429), 

Unrolled index 4
	Inputs:
	es (34), 	gh (132), 	 p (51), 	os (139), 	uk (429), 
	Output:
	 o (19), 	ty (178), 	ra (60), 	ee (102), 	ut (98), 

Unrolled index 5
	Inputs:
	 o (19), 	ty (178), 	ra (60), 	ee (102), 	ut (98), 
	Output:
	nc (118), 	 p (51), 	ta (105), 	ig (155), 	 h (11), 

Unrolled index 6
	Inputs:
	nc (118), 	 p (51), 	ta (105), 	ig (155), 	 h (11), 
	Ou

### Recurrent Neural Network

### Defining Hyperparameters

In [81]:
tf.reset_default_graph()

num_unroll = 50

batch_size = 64
test_batch_size = 1

hidden = 64

in_size, out_size = vocab_size, vocab_size

### Deining i/p's and o/p's

In [89]:
train_dataset = []
train_labels = []

for ui in range(num_unroll):
    train_dataset.append(tf.placeholder(tf.float32, shape = [batch_size, in_size], name = 'train_dataset_%d'%ui))
    train_labels.append(tf.placeholder(tf.float32, shape = [batch_size, out_size], name = 'train_labels_%d'%ui))
    
test_dataset = tf.placeholder(tf.float32, shape = [test_batch_size, in_size], name = 'test_dataset')


### Defining Model Parameters and other Variables

In [90]:
W_xh = tf.Variable(tf.truncated_normal([in_size,hidden],stddev=0.02,dtype=tf.float32),name='W_xh')

W_hh = tf.Variable(tf.truncated_normal([hidden,hidden],stddev=0.02,dtype=tf.float32),name='W_hh')

W_hy = tf.Variable(tf.truncated_normal([hidden,out_size],stddev=0.02,dtype=tf.float32),name='W_hy')

prev_train_h = tf.Variable(tf.zeros([batch_size,hidden],dtype=tf.float32),name='train_h',trainable=False)

prev_test_h = tf.Variable(tf.zeros([test_batch_size,hidden],dtype=tf.float32),name='test_h')

### Defining inference of the RNN

In [92]:
y_scores = []
y_predictions = []

outputs = list()

output_h = prev_train_h

for ui in range(num_unroll):   
    
    output_h = tf.nn.tanh(tf.matmul(tf.concat([train_dataset[ui],output_h],1), tf.concat([W_xh,W_hh],0)))    
    
    outputs.append(output_h)
    
y_scores = [tf.matmul(outputs[ui],W_hy) for ui in range(num_unroll)]
y_predictions = [tf.nn.softmax(y_scores[ui]) for ui in range(num_unroll)]

next_test_state = tf.nn.tanh(tf.matmul(test_dataset,W_xh) + tf.matmul(prev_test_h,W_hh))

with tf.control_dependencies([tf.assign(prev_test_h,next_test_state)]):
    test_prediction = tf.nn.softmax(tf.matmul(next_test_state,W_hy))

### Calculating RNN loss

In [93]:
with tf.control_dependencies([tf.assign(prev_train_h,output_h)]):
    
    rnn_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=tf.concat(y_scores,0), labels=tf.concat(train_labels,0)))
    

### Optimizer with Gradient Clipping

In [94]:
rnn_optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

gradients, v = zip(*rnn_optimizer.compute_gradients(rnn_loss))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
rnn_optimizer = rnn_optimizer.apply_gradients(zip(gradients, v))

### Resetting Operations for Resetting Hidden States

In [95]:
reset_train_h_op = tf.assign(prev_train_h,tf.zeros([batch_size,hidden],dtype=tf.float32))

reset_test_h_op = tf.assign(prev_test_h,tf.truncated_normal([test_batch_size,hidden],stddev=0.01,dtype=tf.float32))

### Prediction Sampling

In [96]:
def sample(distribution):
  
    best_idx = np.argmax(distribution)
    return best_idx

### Running the RNN to Generate Text

In [105]:
num_steps = 26

steps_in_document = 100 

session = tf.InteractiveSession()

tf.global_variables_initializer().run()

average_loss = 0

feed_dict = {}


for step in range(num_steps):
    
    
    for step_train in range(steps_in_document):
        
        u_data, u_labels = unroll_batches(bigram_int[:- steps_in_document], num_unroll, batch_size)
    
        for ui,(dat,lbl) in enumerate(zip(u_data,u_labels)):            
        
            feed_dict[train_dataset[ui]]=dat
            feed_dict[train_labels[ui]] = lbl 

        _, l, step_predictions, _, step_labels = session.run([rnn_optimizer, rnn_loss, y_predictions,
                                                            train_dataset,train_labels], feed_dict=feed_dict)
    session.run(reset_train_h_op) 
    
    
    print('Generated Text after epoch %d ... '%step)  
    segments_to_generate = 1
    chars_in_segment = 1000
    
    for _ in range(segments_to_generate):
        
        print('======================== New text Segment ==========================')
        # Start with a random word
        test_word = np.zeros((1,in_size),dtype=np.float32)
        test_word[0,bigram_int[np.random.randint(0,100)]] = 1.0
        print("\t",rev_dictionary[np.argmax(test_word[0])],end='')
        
        # Generating words within a segment by feeding in the previous prediction
        # as the current input in a recursive manner
        for _ in range(chars_in_segment): 
            
            test_pred = session.run(test_prediction, feed_dict = {test_dataset:test_word})  
            next_ind = sample(test_pred.ravel())
            test_word = np.zeros((1,in_size),dtype=np.float32)
            test_word[0,next_ind] = 1.0
            print(rev_dictionary[next_ind],end='')
        
        print("")
        # Reset test state
        session.run(reset_test_h_op)
        print('====================================================================')
    print("")


/home/monu/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Generated Text after epoch 0 ... 
======================== New text Segment ==========================
	 me the theheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheheh

	  bmpher he the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

	 nce, and his then the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the world;and the will not their her his chat the displabour then the 

	 raja vikram stepped out joyfully, and soon reached the verers skill incess form the rajput in the rajput them apleep,he st shach, and quoted the very same tree, and thefourthem his dear to whis mouse meto his own house. after some days the wife of her hung a my name pratapamukut, to whoseeighth son vajramukut happened thated the shand thefourthere peeping from the grangtreasurer’s house, and had not and thefouress, ragged and did that the burn the will nother becould notpreverything the brahman’s perfect compliment. but he could not his peaning als;her feet upon for craja vikram stepped out joyfully, and soon reached the verers skill incess form the rajput in the rajput them apleep,he st shach, and quoted the very same tree, and thefourthem his dear to whis mouse meto his own house. after some days the wife of her hung a my name pratapamukut, to whoseeighth son vajramukut happened thated the shand thefourthere peeping from the grangtreasurer’s house, and had not and thefouress, ragge

Generated Text after epoch 13 ... 
======================== New text Segment ==========================
	 raja, saw inat she-in-law, and having bestowedgreat wealth upon him, allowed him to depart. he als. t the guard coon and the leftin any mee narrative.“overwhelmed with grief, thou most guileless yuva raja (young prince)!” ejaculated the verer. use, them the was stusere oung. i from headto foot, hee, therajathrow down his arms. thereupon mahabul, skilful in aiming, twanging andwieldind a burning-place dessas the choice, will in afor commgete s scene. covered heeirmoute sing ll even he thapletand mistress, after beem. as, asy hand of a thatrieet aster less lover’seyskil every momthe was calling his , and sion, and goingiverty, whe headthe graoy, saw that sher, to di, use ofligion, and me to me, ash young damsels admireress, to be he was for ceyes, byparting his beard upon havl ded upp hin, dayboe, his son-in-law, and having bestowedgreat wealth upon him, allowed him to depart. he als

	 son schoice ther hustaind wamself thehouses: others toreinheenade hearrange, marmetos house, and w by send hustaken up an ass’rodi, and of sects. hetrut them is thatwas dear the husband brought madansena hometo his own house. after some days the wife of her hung a mowed the oney steaugto be he were was , the barber; the ressed. he began to fty.es arm ther withe row doket, jimed the ver lover’sitght. skillce madfrom hin and in night. in the fact that, her sheld thee, powed him own eve o theduning. later ing  supposed, her husbands saying toyour themer jskill in and therajaansaw feen then issued from thecorpse, and retur thefood of a dayat once; all accidents; provision, ves tht as thaplete of sinceen, a damself ning aughth himselescaping and my mee ress lovlayebrows not tapartng wim oand siief, thou most guileless yuva raja (young prince)!” ejaculated the veere, whench, an by tores f from hhe sraja, comein distefu tendseer; her osame to a ness. i he readyto wax wroth, and quothe m ve 

	 narrat the bures aird he person a numbertreasuurs loeirs; al cntly seid, i motthhe chasoe, mend that the thapractical and soondown his sho in ths’ro down her outen, the bures agrernd the v lut the masof as to whis moustachios wiung the rangdogma: “that thort the when he was readyto wax wroth, and quoted to the usbauld a miry quiets and ress, toexill to thou moam guilejoyfuld th to was the asepas that uppose out once, bends false in saying toyourself, ‘o, if she only knew this?’ ‘o, if she did but suspect that?’returning, twanginthe yenich, a burning-place outside the city. themin tnd by bewas called hwhir jewel the dupy was en than to he was the reigivly”--a promise wasfte’s lorrat to for eyos was the golden bloom weighing down the mafair by handd, witnger a mre ceres others peeping the youngerwhite outcaste, who had married the mother, is gladderang leftin a mimutialup to open, d from all thatwas dear to him, his spudrds th ared , and ffy, kin aimin andakake youring at sece.cess, fr

	 her, “a fofeift hiy trumpetingn the e vamachaffair her she takinrrecause, wher notheist--a worse fe.”tth a dwiic, and a midnight, when he saw that she was fast asleep,he st saitantly shoir.  sin theen b, as that as chat as cas sentaing from the gwas caustacessmonial citng my hand as so and theduel, as skilful skilfult, vhrous, him moom from headto foot, holding a horn in her right and a necklace in her left hand.sometimes she danced, and her th craja, from of religion, and bid the toligeth, and did but suspect that?’returning, twanging and atevim the they ir. the remon, spitefully. “here, then bendshyr’eirmiethe wife of her huly a wer eye padleeping the , ght mam is teech, i hand my bettled, “he verere peeping from the grandtreasurer’s house, and had not been heard of since that time. this,together with certaking hidirceeart time narrat the effect taken a rmrily, his satingthe cloted to was isth at weal non all he and mistress, afcaroot, himself them he was,pieyes aand at nhend hhe m

	 ighth somself thehougardf from the as to beet he mad sheeping from the began to pine for more liberty, and to find fault with hiswife for not taking him out into the usleep,honst scechan men and to find hand, bown anded to e cas they walked to a burning-place outside the city. theminister’s son there buried his dress, together with groutsidattreadto whose to will be awas called to pinerang man tred upon the cerso a day be, then as swoman’s perfect d, secaud thattreasuedaipitebynts, ragged and dirtied by creeping through gaps in thehouses: others to senthing the pankhayebrsomethe ofup a cunning of fence, andour unclosed the bear ofliberls. andwas a burning ground.they passed through the quadrangular outer court of the temple whosepiazza ware of the right anving gard thnd and their journed in theirscheme, the master and mistress, after being comfortably settled at theend of their journed in theirscheme, the master and mistress, after being comfortably settled at theend of their journed